# NEWEST VER

In [ ]:
# pip install earthengine-api --upgrade

  0%|          | 0/10 [21:07:24<?, ?epoch/s]

In [ ]:
# pip install geemap

**remember to re-auth every time you want to upload**

In [ ]:
import ee
print()
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Adeu5BUPcUnAYmWXR2vm7dvYeWmTM_SvvbDpbp0Qmev8e3Y9WZ1Wi3LJtcM



Successfully saved authorization token.


In [ ]:
import numpy as np
import pandas as pd

import geemap
import folium

In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.run_functions_eagerly(True)

Define datasets

In [ ]:
feature_list = []
# Fire label
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/FIRMS
fire_map_info = {'name':'FIRMS', 'band' :'T21', 'min': 300, 'max': 509.29}
# MODIS Normalized difference vegetation index NDVI dataset
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_MOD09GA_006_NDVI
veg_map_info = {'name':'MODIS/MOD09GA_006_NDVI', 'band' :'NDVI', 'min': -1, 'max': 1}
feature_list.append(veg_map_info)
# MODIS Leaf Area Index/FPAR 4-Day Global 500m
# 2002-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD15A3H
# upper limit is 20+, so set max = 30
leaf_map_info = {'name':'MODIS/061/MCD15A3H', 'band' :'Lai', 'min': 0, 'max': 30}
feature_list.append(leaf_map_info)
# FLDAS: Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System
# 1982-2023
# https://developers.google.com/earth-engine/datasets/catalog/NASA_FLDAS_NOAH01_C_GL_M_V001
soil_moist_map_info = {'name':'NASA/FLDAS/NOAH01/C/GL/M/V001', 'band' :'SoilMoi00_10cm_tavg', 'min': 0, 'max': 1}
feature_list.append(soil_moist_map_info)
# Terra Land Surface Temperature and Emissivity Daily Global 1km
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1
temp_map_info = {'name':'MODIS/061/MOD11A1', 'band' :'LST_Day_1km', 'min': 13658, 'max': 15658}
feature_list.append(temp_map_info)
# ERA5 Monthly Aggregates dataset
# 1979-2020
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY
precipitation_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'total_precipitation', 'min': 0, 'max': 0.4}
feature_list.append(precipitation_map_info)
wind_speed_u_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'u_component_of_wind_10m', 'min': -8.7, 'max': 8.7}
feature_list.append(wind_speed_u_map_info)
wind_speed_v_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'v_component_of_wind_10m', 'min': -6.8, 'max': 6.8}
feature_list.append(wind_speed_v_map_info)

# Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3
# 2015-end2019
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global
land_cover_info = {'name':"COPERNICUS/Landcover/100m/Proba-V-C3/Global", 'band': 'discrete_classification', 'min': 0, 'max': 200}
# feature_list.append(wind_speed_v_map_info)
# print(feature_list)
print(len(feature_list))
print(type(feature_list))

7
<class 'list'>


Define label and feature bands

In [ ]:
LABEL = fire_map_info['band']
BANDS = []
for feature in feature_list:
  BANDS += [feature['band']]
FEATURE_NAMES = BANDS + [LABEL]
print(FEATURE_NAMES)

['NDVI', 'Lai', 'SoilMoi00_10cm_tavg', 'LST_Day_1km', 'total_precipitation', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'T21']


Define area of interest and time range

In [ ]:
# PROJECT = 'big-bliss-393615'
# DATA_BUCKET = 'big-bliss-393615-6140-data-bucket'
# OUTPUT_BUCKET = 'big-bliss-393615-6140-output-bucket'
PROJECT = 'ee-my-char'
DATA_BUCKET = '6140-data-bucket'
OUTPUT_BUCKET = '6140-output-bucket'
# Specify names locations for outputs in Cloud Storage.
FOLDER = 'fcnn-demo'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'
# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
# KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURE_NAMES]
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))
# Specify model training parameters.
BATCH_SIZE = 64
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

In [ ]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Canada = countries.filter(ee.Filter.eq('country_na', 'Canada')).geometry() # <class 'ee.geometry.Geometry'>
America = countries.filter(ee.Filter.eq('country_na', 'United States')).geometry()

In [ ]:
train_start_year = 2015
train_end_year = 2015
train_start_month = 4
train_end_month = 4
# predict fire 4 weeks in advance
fire_pred_range = 2
# every 2 weeks get one image
sample_gap = 2
train_region_geometry = America
pred_region_geometry = Canada

- How many time slices do we have Calculated in getTrainImageCollection

In [ ]:
total_temporal_slices = 0  # Initialize the total number of temporal slices.
temporal_list = []  # Initialize the list to store temporal date strings.

# Loop through the years and months within the specified training time range.
for yr in range(train_start_year, train_end_year + 1):
    for mo in range(train_start_month, train_end_month + 1):
        total_temporal_slices += sample_gap  # Increment the total time slices by 'sample_gap'.
        temporal_list.append(f'{yr}-{mo:02d}-01')  # Add the first day of the month to the list.
        temporal_list.append(f'{yr}-{mo:02d}-15')  # Add the fifteenth day of the month to the list.
print(temporal_list)

['2015-04-01', '2015-04-15']


If 'is_label', advance the time range by 'fire_pred_range', e.g. we use first 0-15 as feature and 16-30 as label

In [ ]:
# Helper function avoid repeating
def getImageCollection(dataset, bounds, band):
  return ee.ImageCollection(dataset).filterBounds(bounds).select(band)

def getTrainImageCollection(map_info, is_label=False):
  img_collection = getImageCollection(map_info['name'],train_region_geometry, map_info['band']) # Get the image collection based on the provided map_info.
  # print(img_collection.size().getInfo())
  img_list = ee.List([])

  advance_range = 0 if not is_label else fire_pred_range
  for yr in range(train_start_year, train_end_year+1):
    for mo in range(train_start_month, train_end_month + 1):
      # first half of a month
      start = ee.Date(f'{yr}-{mo:02d}-01').advance(advance_range, 'week')
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end) # Filter the image collection based on the date range.
      if not is_label:
        med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
      if is_label:
        med = date_filtered.median().unmask(0).divide(300).round().byte() # credit: david
      print(med.getInfo())
      img_list = img_list.add(med)

      # second half of a month
      start = end
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end) # Filter the image collection based on the date range.
      if date_filtered.size().getInfo() > 0:  # Check if there are images in the second half of the month.
        # print("value exists from 15")
        if not is_label:
          med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
        if is_label:
          med = date_filtered.median().unmask(0).divide(300).round().byte() # credit: david
        print(med.getInfo())
        
        # med = date_filtered.median().unmask(0).unitScale(map_info['min'], map_info['max'])
        # print(f'med type: {type(med)}')
      img_list = img_list.add(med)

  # print(img_list.size().getInfo())
  return img_list

Each element in feature_img_lists is a temporal imageCollection of a single band

In [ ]:
# Label
firms = getTrainImageCollection(fire_map_info, True)

{'type': 'Image', 'bands': [{'id': 'T21', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'T21', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


`firms_mask_lists` stores firms info in temporal order
each element in the list is a list of 2 elements, the first is `has_fire_mask`, the second is `no_fire_mask`

In [ ]:
def get_firms_mask(firms):
    firms_masks = []
    for idx in range(firms.size().getInfo()):
        img = ee.Image(firms.get(idx))
        has_fire_mask = img.select('T21').eq(1)    
        no_fire_mask = img.select('T21').eq(0) 
        cur_time_slice_mask_set = []
        cur_time_slice_mask_set.append(has_fire_mask)
        cur_time_slice_mask_set.append(no_fire_mask)
        firms_masks.append(cur_time_slice_mask_set)
    return firms_masks
firms_mask_lists = get_firms_mask(firms)

In [ ]:
feature_img_lists = []
for feature in feature_list:
  img_list = getTrainImageCollection(feature)
  feature_img_lists.append(img_list)
# print(type(feature_img_lists))
# print(feature_img_lists)

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'Lai', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'Lai', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'SoilMoi00_10cm_tavg', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image', 'b

In [ ]:
# # `feature_img_masked_lists` stores lists of features
# # each list is ordered in temporal order
# # within each list, stores a list of 2 elements, the first is feature with fire mask, the second is feature with no fire mask
# def get_features_with_firms_mask(feature_img_lists, firms_mask_list):
#     feature_img_masked_lists = []
#     for cur_feature_all_time in feature_img_lists:
#         cur_feature_masked_list = []
#         for cur_time_slice in range(cur_feature_all_time.size().getInfo()):
#             img = ee.Image(cur_time_slice)
#             img_mask_with_fire = img.updateMask(firms_mask_list[cur_time_slice][0])
#             img_mask_no_fire = img.updateMask(firms_mask_list[cur_time_slice][1])
#             cur_time_slice_mask_set = []
#             cur_time_slice_mask_set.append(img_mask_with_fire)
#             cur_time_slice_mask_set.append(img_mask_no_fire)
#             cur_feature_masked_list.append(cur_time_slice_mask_set)
#         feature_img_masked_lists.append(cur_feature_masked_list)
#     return feature_img_masked_lists    

# feature_img_masked_lists = get_features_with_firms_mask(feature_img_lists, firms_mask_lists)
# print(feature_img_masked_lists[0][0].getInfo())

AttributeError: 'list' object has no attribute 'getInfo'

export sample unmask fire as csv

In [ ]:
# # date_start = '2015-04-01'
# # date_end = '2015-12-01'
# # dataset = ee.ImageCollection('FIRMS').filter(
# #     ee.Filter.date(date_start, date_end))
# # fires = dataset.select('T21').first().unmask(0)
# Map = geemap.Map(center=[47.295, -119.086], zoom=6)
# # Map
# fires = ee.Image(firms.get(1))
# print(type(fires))

# fire_values = fires.reduceRegion(
#     reducer=ee.Reducer.toList(),
#     geometry=Map.getBounds(False),
#     scale=1000,  # The scale from the image's 'crs_transform'
#     maxPixels=90000000
# )
# fire_values.getInfo()
# df = pd.DataFrame(fire_values.getInfo())
# df.to_csv('fires_data.csv', index=False)

test unmask

In [ ]:
# import pandas as pd

# df = pd.read_csv('fires_data.csv')

# unique_values_count = df.nunique()

# unique_values = {}
# for column in df.columns:
#     unique_values[column] = df[column].unique()

#     print(f"Unique values in column '{column}':")
#     print(unique_values[column])
#     print(f"Number of unique values: {len(unique_values[column])}")

#     value_counts = df[column].value_counts()
#     print(f"Value counts:")
#     print(value_counts)
#     print()

Debug function: get min & max by iterating through all pixels

In [ ]:
# @title
# For debug, can skip
def get_min(gee_image, key):
  min_dic = gee_image.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry = Canada.geometry(),
    crs =  'EPSG:4326',
    crsTransform = [1, 0, 0, 0, 1, 0]
  )
  return min_dic.getInfo().get(key)
 
def get_max(gee_image, key):
  max_dic = gee_image.reduceRegion(
    reducer= ee.Reducer.max(),
    geometry = Canada.geometry(),
    crs =  'EPSG:4326',
    crsTransform = [1, 0, 0, 0, 1, 0]
  )
  return max_dic.getInfo().get(key)

Visualize all maps

In [ ]:
# # @title
# # visualization
# # normalized
# firms_vis = firms.median().clip(train_region_geometry)
# vegetation_vis = vegetation.median().clip(train_region_geometry)
# leaf_vis = leaf_index.median().clip(train_region_geometry)
# soil_moist_vis = soil_moist.median().clip(train_region_geometry)
# precipitation_vis = precipitation.median().clip(train_region_geometry)
# temperature_vis = temperature.median().clip(train_region_geometry)
# wind_speed_vis = wind_speed_u.median().clip(train_region_geometry).cat(wind_speed_v.median().clip(train_region_geometry))

# Map = geemap.Map()
# ROI = ee.Geometry.Point([-122.6544033634249, 48.99021037738678]).buffer(100000);

# Map.addLayer(firms_vis, {'min': 0, 'max': 1, 'palette': ['LightYellow','Yellow', 'Gold', 'Orange', 'Tomato', 'Red']}, 'FIRMS')
# Map.addLayer(vegetation_vis, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, 'vegetation')
# Map.addLayer(leaf_vis, {'min': 0, 'max': 1, 'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']}, 'leaf index')
# Map.addLayer(soil_moist_vis, {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'soil moisture')
# Map.addLayer(wind_speed_vis, {'min': 0, 'max': 1, 'palette': ['blue', 'red']}, 'wind speed')
# Map.addLayer(precipitation_vis, {'min': 0,
#   'max': 1,
#    'palette': [
#     '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff',
#     '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00',
#     'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff',
#   ]}, 'precipitatin')

# Map.addLayer(temperature_vis, {  'min': 0,
#   'max': 1,
#   'palette': [
#     '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
#     '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
#     '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
#     'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
#     'ff0000', 'de0101', 'c21301', 'a71001', '911003'
#   ]}, 'temperature')

# Map.addLayerControl()
# Map

In [ ]:
# # @title
# Map.user_roi

Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band. This is a key step that bears emphasis: to export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.

In [ ]:
list_1d = ee.List.repeat(1, KERNEL_SIZE)
list_2d = ee.List.repeat(list_1d, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, list_2d)

In [ ]:
def mask_stack_kernel(feature_stack, mask, kernel):
    # mask
    masked_feature_stack = feature_stack.updateMask(mask)
    print(f'debug {masked_feature_stack.getInfo()}')
    # stack, no need to do because mask step adds the mask band to result
    # masked_feature_stack = ee.Image.cat([masked_feature_stack, mask])
    # kernel
    arrays = masked_feature_stack.neighborhoodToArray(kernel)
    return arrays

def get_feature_stack(kernel, feature_img_lists, firms_mask_list):
    list_arrays_has_fire = []
    list_arrays_no_fire = []
    for tm in range(total_temporal_slices): 
        # Get the image for the current temporal slice from feature_img_lists
        featureStack = feature_img_lists[0].get(tm)
        # Concatenate all the bands sampled from different images into featureStack
        for idx in range(1, len(feature_img_lists)):
            # Combine all bands sampled from the same time to featureStack
            featureStack = ee.Image.cat([featureStack, feature_img_lists[idx].get(tm)]).float()
        
        arrays_has_fire = mask_stack_kernel(featureStack, firms_mask_list[tm][0], kernel)
        arrays_no_fire = mask_stack_kernel(featureStack, firms_mask_list[tm][1], kernel)
        print(f'time = {tm}:\nhas fire:{arrays_has_fire.getInfo()}\nno fire:{arrays_no_fire.getInfo()} ')
        list_arrays_has_fire += [arrays_has_fire]
        list_arrays_no_fire += [arrays_no_fire]
    
    return list_arrays_has_fire, list_arrays_no_fire

list_arrays_has_fire, list_arrays_no_fire = get_feature_stack(kernel, feature_img_lists, firms_mask_lists)
print(f'{len(list_arrays_has_fire)}, {len(list_arrays_no_fire)}')

debug {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'Lai', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SoilMoi00_10cm_tavg', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'u_component_of_wind_10m', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'v_component_of_wind_10m', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transfor

feature_img_lists
NDVI:[4-1, ..., 12-1]
LAI :[4-1, ..., 12-1]

list_arrays [4-1, ...,]

In [ ]:
# list = ee.List.repeat(1, KERNEL_SIZE)
# lists = ee.List.repeat(list, KERNEL_SIZE)
# kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

# list_arrays = []

# for tm in range(total_temporal_slices): # 2
#   # Get the image for the current temporal slice from feature_img_lists
#   featureStack = feature_img_lists[0].get(tm)
#   # Concatenate all the bands sampled from different images into featureStack
#   for idx in range(1, len(feature_img_lists)):
#     # Combine all bands sampled from the same time to featureStack
#     featureStack = ee.Image.cat([featureStack, feature_img_lists[idx].get(tm)]).float()
#   # Concatenate the FIRMS band to featureStack and convert it to float
#   featureStack = ee.Image.cat([featureStack, firms.get(tm)])
#   # Apply neighborhood operations using the kernel and convert to arrays
#   arrays = featureStack.neighborhoodToArray(kernel)
#   # Add the array to the list_arrays for the current temporal slice
#   list_arrays += [arrays]

In [ ]:
# # Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
# states = ee.FeatureCollection("TIGER/2016/States")
# # Get the distinct values of the 'NAME' property
# distinct_names = states.distinct('NAME').aggregate_array('NAME')
# # Convert the distinct names to a Python list
# all_state_names = distinct_names.getInfo()
# # Print all state names
# print(all_state_names)

Get polygons for training and evaluation

In [ ]:
# Create an empty FeatureCollection to store the training polygons
trainingPolys = ee.FeatureCollection([])

# Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
states = ee.FeatureCollection("TIGER/2016/States")

# List of state names to be used for training
# state_list = ['California', 'Oregon','Montana','Washington','Arizona', 'Idaho','Texas','Kansas','Nevada','New Mexico']
state_list = ['California', 'Oregon', 'Texas', 'Arizona', 'Nevada', 'New Mexico']
# Iterate through each state in 'state_list'
for state in state_list:
    # Merge the geometry of the state from 'states' into 'trainingPolys'
    trainingPolys = trainingPolys.merge(states.filter(ee.Filter.eq('NAME', state)).geometry())

# Convert the 'trainingPolys' FeatureCollection to a List
trainingPolysList = trainingPolys.toList(trainingPolys.size())
# print(type(trainingPolysList))

In [ ]:
# Create an empty FeatureCollection to store the training polygons
valPolys = ee.FeatureCollection([])

# Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
states = ee.FeatureCollection("TIGER/2016/States")

# List of state names to be used for training
# state_list = ['Oklahoma', 'Colorado']
state_list = ['Wyoming', 'Idaho', 'Alaska', 'Washington', 'New Jersey', 'Pennsylvania']

# Iterate through each state in 'state_list'
for state in state_list:
    # Merge the geometry of the state from 'states' into 'trainingPolys'
    valPolys = valPolys.merge(states.filter(ee.Filter.eq('NAME', state)).geometry())
    # print(states.filter(ee.Filter.eq('NAME', state)).geometry().getInfo())

# Convert the 'trainingPolys' FeatureCollection to a List
valPolysList = valPolys.toList(valPolys.size())

# print(valPolysList.size().getInfo())
print(type(valPolysList))
# # print(valPolys.geometry().getInfo())
# print(valPolys.get(0))

<class 'ee.ee_list.List'>


Visualize training and evaluation regions on map

In [ ]:
Map = geemap.Map()
Map.addLayer(trainingPolys, {'color': 'FF0000'})
Map.addLayer(valPolys, {'color': '00FF00'})
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

- Export training data

In [ ]:
SAMPLE_NUM_IN_POLYGON = 200 # Number of shards in each polygon.
SAMPLE_PIXEL_IN_POLYGON = 2000 # Total sample pixels in each polygon.

In [ ]:
# Sizes of the training and evaluation datasets.
TRAIN_SIZE = len(temporal_list) * trainingPolysList.size().getInfo() * SAMPLE_NUM_IN_POLYGON
EVAL_SIZE = len(temporal_list) * valPolysList.size().getInfo() * SAMPLE_NUM_IN_POLYGON
# print(TRAIN_SIZE)

In [ ]:
SAMPLE_RESOLUTION = 1000 # 1km

In [ ]:
def export_data(file_prefix, sample_poly_list, sample_source_list,file_suffix, num_sample_pixel, early_exit_time_idx = None):
    for idx, time_str in enumerate(temporal_list): # 18, Iterate through each time string in 'temporal_list'
        if idx == early_exit_time_idx:
            break
        for g in range(sample_poly_list.size().getInfo()): # 10, Iterate through each geometry in 'sample_poly_list'
            geomSample = ee.FeatureCollection([])
            for i in range(SAMPLE_NUM_IN_POLYGON): # 200
                sample = sample_source_list[idx].sample(
                    region = ee.Feature(sample_poly_list.get(g)).geometry(),
                    scale = SAMPLE_RESOLUTION, # 1km
                    numPixels = num_sample_pixel / SAMPLE_NUM_IN_POLYGON, # 200/200, 
                    seed = i,
                    tileScale = 8 # make your process faster
                )
                geomSample = geomSample.merge(sample)
            desc = f'{file_prefix}_{time_str}_{str(g)}_{file_suffix}'
            print(desc)
            task = ee.batch.Export.table.toCloudStorage(
              collection = geomSample,
              description = desc,
              bucket = DATA_BUCKET,
              fileNamePrefix = FOLDER + '/' + desc,
              fileFormat = 'TFRecord',
              selectors = FEATURE_NAMES
            )
            task.start()

In [ ]:
NUM_PIXEL_SAMPLE_NO_FIRE = 200
NUM_PIXEL_SAMPLE_FIRE = NUM_PIXEL_SAMPLE_NO_FIRE * 1000
print(NUM_PIXEL_SAMPLE_FIRE, NUM_PIXEL_SAMPLE_NO_FIRE)

200000 200


In [ ]:
export_data(TRAINING_BASE, trainingPolysList, list_arrays_has_fire, 'fire', NUM_PIXEL_SAMPLE_FIRE)   
export_data(TRAINING_BASE, trainingPolysList, list_arrays_no_fire, 'nofire', NUM_PIXEL_SAMPLE_NO_FIRE)           

training_patches_2015-04-01_0_fire
training_patches_2015-04-01_1_fire
training_patches_2015-04-01_2_fire
training_patches_2015-04-01_3_fire
training_patches_2015-04-01_4_fire
training_patches_2015-04-01_5_fire
training_patches_2015-04-15_0_fire
training_patches_2015-04-15_1_fire
training_patches_2015-04-15_2_fire
training_patches_2015-04-15_3_fire
training_patches_2015-04-15_4_fire
training_patches_2015-04-15_5_fire
training_patches_2015-04-01_0_nofire
training_patches_2015-04-01_1_nofire
training_patches_2015-04-01_2_nofire
training_patches_2015-04-01_3_nofire
training_patches_2015-04-01_4_nofire
training_patches_2015-04-01_5_nofire
training_patches_2015-04-15_0_nofire
training_patches_2015-04-15_1_nofire
training_patches_2015-04-15_2_nofire
training_patches_2015-04-15_3_nofire
training_patches_2015-04-15_4_nofire
training_patches_2015-04-15_5_nofire


In [ ]:
export_data(EVAL_BASE, valPolysList, list_arrays_has_fire, 'fire', NUM_PIXEL_SAMPLE_FIRE)
export_data(EVAL_BASE, valPolysList, list_arrays_no_fire, 'nofire', NUM_PIXEL_SAMPLE_NO_FIRE)           

eval_patches_2015-04-01_0_fire
eval_patches_2015-04-01_1_fire
eval_patches_2015-04-01_2_fire
eval_patches_2015-04-01_3_fire
eval_patches_2015-04-01_4_fire
eval_patches_2015-04-01_5_fire
eval_patches_2015-04-15_0_fire
eval_patches_2015-04-15_1_fire
eval_patches_2015-04-15_2_fire
eval_patches_2015-04-15_3_fire
eval_patches_2015-04-15_4_fire
eval_patches_2015-04-15_5_fire
eval_patches_2015-04-01_0_nofire
eval_patches_2015-04-01_1_nofire
eval_patches_2015-04-01_2_nofire
eval_patches_2015-04-01_3_nofire
eval_patches_2015-04-01_4_nofire
eval_patches_2015-04-01_5_nofire
eval_patches_2015-04-15_0_nofire
eval_patches_2015-04-15_1_nofire
eval_patches_2015-04-15_2_nofire
eval_patches_2015-04-15_3_nofire
eval_patches_2015-04-15_4_nofire
eval_patches_2015-04-15_5_nofire


Test: use stratified result of firms as sampleRegion input for features

In [ ]:
# firms_stratified_time_result = [] # list of every time slice
# for i in range(firms.size().getInfo()): # firms.size() == time slices
#     cur_firm = ee.Image(firms.get(i))
#     firms_stratified_region_result = [] # list of every region in trainingPolysList
    
#     for g in range(trainingPolysList.size().getInfo()):
#         firms_stratified_feature_collection = ee.FeatureCollection([]) # feature collection at current region and time slice
#         for sample_idx in range(SAMPLE_NUM_IN_POLYGON): # Loop through each sample index for the current polygon
#             sample = cur_firm.stratifiedSample(
#                 numPoints=SAMPLE_PIXEL_IN_POLYGON / SAMPLE_NUM_IN_POLYGON, # 5
#                 classBand='T21',
#                 region = ee.Feature(trainingPolysList.get(g)).geometry(),
#                 scale = SAMPLE_RESOLUTION, # 1km
#                 seed = i,
#                 tileScale = 8, # make your process faster
#                 geometries = True
#             )
#             firms_stratified_feature_collection = firms_stratified_feature_collection.merge(sample)
#         # print(firms_stratified_feature_collection.distinct("T21").getInfo())
#         # print(firms_stratified_feature_collection.size().getInfo())
#         firms_stratified_region_result.append(firms_stratified_feature_collection)
#     firms_stratified_time_result.append(firms_stratified_region_result)
    
# # print(type(firms_stratified_time_result[0][0])) # <class 'ee.featurecollection.FeatureCollection'>

In [ ]:
# for time_idx, time_slice_result in enumerate(firms_stratified_time_result): #2
#     # print(len(time_slice_result))
#     for region_idx, region_result in enumerate(time_slice_result): # 2
#         print(region_result.size().getInfo()) # featurecollection
#         print(type( list_arrays[time_idx]))
#         geomSample = list_arrays[time_idx].sampleRegions(
#             collection = region_result,
#             properties = ee.List(['T21']),
#             scale = SAMPLE_RESOLUTION, # 1km
#             tileScale = 8 # make your process faster
#           )
#         # print(type(geomSample))
#         desc = f'sampleRegion_{time_idx}_{region_idx}'
#         # print(desc)
#         task = ee.batch.Export.table.toCloudStorage(
#           collection = geomSample,
#           description = desc,
#           bucket = DATA_BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = FEATURE_NAMES
#         )
#         # task.start()

sample and export train

stratify sample

- export eval

Load the data exported
去到bucket已经能看到这个file下载好了! 在那边! 接下来就要读 **Text fett markieren**

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  # print(f'Example Proto: {example_proto}')
  parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
  # print(f'Parsed features: {parsed_features}')
  # print()
  return parsed_features

def to_tuple(inputs, deb=True):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURE_NAMES]
  stacked = tf.stack(inputsList, axis=0)

  if deb:
    print(f'to_tuple inputsList {inputsList}')
    print(f'to_tuple len list {len(inputsList)}')
    print(f'to_tuple stacked shape {stacked.shape}')

  # Convert from CHW to HWC
  # See https://caffe2.ai/docs/tutorial-image-pre-processing.html
  stacked = tf.transpose(stacked, [1, 2, 0])

  if deb:
    print(f'to_tuple after reshape {stacked.shape}')
    print(f'length of BANDS??? {len(BANDS)}')
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]

In [ ]:
# visualize tfrecord
import matplotlib.pyplot as plt

def visualize_dataset(dataset):
  for batch in dataset.take(3):
    # print(f'Batch keys: {batch.keys()}')
    # print(f'Batch data types: {[type(v.numpy()) for v in batch.values()]}')
    f, axarr = plt.subplots(4, 2)
    rows = np.repeat([0, 1, 2, 3], 2)
    cols = np.repeat([[0, 1]], 4, axis=0).ravel()
    for v, r, c in zip(batch.values(), rows, cols):
      axarr[r, c].imshow(v, cmap='gray')

def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  # print(f'get_dataset:parse step1 {dataset}')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  # print(f'get_dataset step1-2 {dataset.element_spec}')
  # for batch in dataset.take(1):
  #   print("Batch keys:", batch.keys())
  #   for key, value in batch.items():
  #       print(f"Key: {key}, Shape: {value.shape}, Data Type: {value.dtype}")
    
  # visualize_dataset(dataset)
  # print(f'get_dataset step2 {dataset}')
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  # print(f'get_dataset step3 {dataset}')
  return dataset

train, eval

fixed problem get training dataset and eval dataset

get new dataset after stratify

In [ ]:
def get_training_dataset(filename_pattern):
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = filename_pattern
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset


def get_eval_dataset(filename_pattern):
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = filename_pattern
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

In [ ]:
training_nofire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*' + '*_nofire*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


In [ ]:
training_fire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*' + '*_fire*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


In [ ]:
eval_fire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*' + '*_fire*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


In [ ]:
eval_nofire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*' + '*_nofire*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


In [ ]:
training = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


In [ ]:
evaluation = get_eval_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*')

to_tuple inputsList [<tf.Tensor 'args_2:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_1:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_3:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_0:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_5:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_6:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_7:0' shape=(256, 256) dtype=float32>, <tf.Tensor 'args_4:0' shape=(256, 256) dtype=float32>]
to_tuple len list 8
to_tuple stacked shape (8, 256, 256)
to_tuple after reshape (256, 256, 8)
length of BANDS??? 7


unet

In [ ]:
filter_1 = 32
filter_2 = 4

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.models as models
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers

In [ ]:
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model(filter_base):
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, filter_base) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, filter_base*2) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, filter_base*4) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, filter_base*8) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, filter_base*16) # 8
	center = conv_block(encoder4_pool, filter_base*32) # center
	decoder4 = decoder_block(center, encoder4, filter_base*16) # 16
	decoder3 = decoder_block(decoder4, encoder3, filter_base*8) # 32
	decoder2 = decoder_block(decoder3, encoder2, filter_base*4) # 64
	decoder1 = decoder_block(decoder2, encoder1, filter_base*2) # 128
	decoder0 = decoder_block(decoder1, encoder0, filter_base) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS],
		run_eagerly=True
		)

	return model

In [ ]:
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.models as models
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers

def conv_block(input_tensor, num_filters):
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same', activation='relu')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    return encoder

def encoder_block(input_tensor, num_filters):
    encoder = conv_block(input_tensor, num_filters)
    encoder_pool = layers.MaxPooling2D((2, 2))(encoder)
    return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
    decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', activation='relu')(input_tensor)
    decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
    decoder = layers.BatchNormalization()(decoder)
    return decoder

def get_model_simplify():
    inputs = layers.Input(shape=[None, None, len(BANDS)])
    encoder0_pool, encoder0 = encoder_block(inputs, 32)
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
    center = conv_block(encoder2_pool, 256)  # Center block with reduced filters
    decoder2 = decoder_block(center, encoder2, 128)
    decoder1 = decoder_block(decoder2, encoder1, 64)
    decoder0 = decoder_block(decoder1, encoder0, 32)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS],
		run_eagerly=True
    )

    return model

In [ ]:
# # taken from https://github.dev/pandyamarut/HalfUnet-ImageSegmentation

# import tensorflow.keras.layers as layers
# import tensorflow.keras.losses as losses
# import tensorflow.keras.models as models
# import tensorflow.keras.metrics as metrics
# import tensorflow.keras.optimizers as optimizers

# def ghost_module(inputs):
#     conv1 = layers.Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(inputs)
#     batch1 = layers.BatchNormalization()(conv1)
#     act1 = layers.Activation('relu')(batch1)
#     conv2 = layers.SeparableConv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(act1)
#     return layers.concatenate([act1, conv2], axis=3)

# def half_unet_model(input_shape=(256, 256, 7)):

#     inputs = layers.Input(shape=input_shape, name="image")
#     x1 = ghost_module(ghost_module(inputs))
#     pool1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
#     x2 = ghost_module(ghost_module(pool1))
#     pool2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
#     x3 = ghost_module(ghost_module(pool2))
#     pool3 = layers.MaxPooling2D(pool_size=(2, 2))(x3)
#     x4 = ghost_module(ghost_module(pool3))
#     pool4 = layers.MaxPooling2D(pool_size=(2, 2))(x4)
#     x5 = ghost_module(ghost_module(pool4))

#     up5 = layers.UpSampling2D(size=(input_shape[0] // x5.shape[1], input_shape[1] // x5.shape[2]))(x5)
#     up4 = layers.UpSampling2D(size=(input_shape[0] // x4.shape[1], input_shape[1] // x4.shape[2]))(x4)
#     up3 = layers.UpSampling2D(size=(input_shape[0] // x3.shape[1], input_shape[1] // x3.shape[2]))(x3)
#     up2 = layers.UpSampling2D(size=(input_shape[0] // x2.shape[1], input_shape[1] // x2.shape[2]))(x2)

#     upScaled = layers.Add()([x1, up2, up3, up4, up5])
#     all_conv = ghost_module(ghost_module(upScaled))
#     final_conv = layers.Conv2D(1, 1, activation='sigmoid')(all_conv)

#     model = models.Model(inputs=[inputs], outputs=[final_conv], name="Half-UNet")
#     model.compile(
# 		optimizer=optimizers.get(OPTIMIZER),
# 		loss=losses.get(LOSS),
# 		metrics=[metrics.get(metric) for metric in METRICS],
# 		run_eagerly=True
#     )
#     return model

sanity check

In [ ]:
# # Sizes of the training and evaluation datasets.
# TRAIN_SIZE = 16000
# EVAL_SIZE = 8000
# BATCH_SIZE = 64
# print(type(training))
# print(EPOCHS)
# print(TRAIN_SIZE)
# print(BATCH_SIZE)
# print(int(TRAIN_SIZE / BATCH_SIZE))
# # print(len(list(training)))
# print(EVAL_SIZE)
# # len(training)
# # Check if GPU is available
# if tf.test.is_gpu_available():
#     # Get the list of available GPU devices
#     gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#     if gpu_devices:
#         # Print the name of the first GPU device
#         print("Using GPU:", gpu_devices[0].name)
#     else:
#         print("No GPU devices available.")
# print("Available GPU devices:")
# print(tf.config.list_physical_devices('GPU'))
# print("Current device used for computation:")
# print(tf.config.list_logical_devices('GPU'))

In [ ]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


modular programming: make sure everything right before training

In [ ]:
stop

NameError: name 'stop' is not defined

model training visualization

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

class TQDMCallback(tf.keras.callbacks.Callback):
    def __init__(self, total_epochs):
        super(TQDMCallback, self).__init__()
        self.total_epochs = total_epochs
        self.pbar = tqdm(total=self.total_epochs, unit='epoch')

    def on_epoch_end(self, epoch, logs=None):
        self.pbar.update(1)
        self.pbar.set_description(f"Epoch {epoch + 1}/{self.total_epochs}, Loss: {logs['loss']:.4f}, "
                                  f"RMSE: {logs['root_mean_squared_error']:.4f}, ")
                                  # f"Val_Loss: {logs['val_loss']:.4f}, "
                                  # f"Val_RMSE: {logs['val_root_mean_squared_error']:.4f}")
    def on_train_end(self, logs=None):
        self.pbar.close()
tqdm_callback = TQDMCallback(total_epochs=EPOCHS)

In [ ]:
# m = get_model(filter_2)
# m = half_unet_model()
m = get_model_simplify()

see model parameter

In [ ]:
m.summary()

Successful fit: 
    model is simplified into containing only a few layers

In [ ]:
history = m.fit(
    x=training,
    epochs=10,
    # steps_per_epoch=int(16000 / 1),
    steps_per_epoch=int(16000 / 64),
    validation_data=evaluation,
    validation_steps=8000,
    callbacks=[tqdm_callback],
)

save model

In [ ]:
m.save('saved_model.h5')

load model

In [ ]:
from tensorflow.keras.models import load_model
m = load_model('saved_model.h5')

In [ ]:
print(history.params)
print(history.history.keys())

visualize

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Training and Validation RMSE')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
batch_size = 64


ev = evaluation.take(100)
# Predict on the evaluation dataset with the specified number of steps
predictions = m.predict(x=ev)

print(len(predictions[0]))
print(len(predictions[0][0]))

In [ ]:
# threshold = 0.5
# predicted_labels = np.where(predictions > threshold, 1, 0)

# true_labels = np.array([label for _, label in ev])

# accuracy = np.sum(predicted_labels == true_labels) / (len(true_labels) * 256*256)
# print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# # Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
# MODEL_DIR = 'gs://ee-docs-demos/fcnn-demo/trainer/model'
# m = tf.keras.models.load_model(MODEL_DIR)
# m.summary()

select image to export

In [ ]:
# print(featureStack.bandNames().getInfo())

In [ ]:
feature_img_lists = []
date_start_str = '2019-07-01'
pred_start = ee.Date(date_start_str)
pred_end = pred_start.advance(4, 'week')

for feature in feature_list:
  img_col = getImageCollection(feature['name'],pred_region_geometry, feature['band'])
  pred_img = img_col.filterDate(pred_start, pred_end).median().unitScale(feature['min'], feature['max'])
  feature_img_lists += [pred_img]
print(feature_img_lists)

In [ ]:
predict_image = feature_img_lists[0]
for idx in range(1, len(feature_img_lists)):
  # Combine all bands sampled from the same time to featureStack
  predict_image = ee.Image.cat([predict_image, feature_img_lists[idx]]).float()
# predict_image

In [ ]:
def doExport(predict_image, out_file_name, kernel_buffer, region):
    """Run the image export task.  Block until complete.
    """
    # Define the export task using the Earth Engine Python API.
    task = ee.batch.Export.image.toCloudStorage(
        image=predict_image,  # Select specific bands from the 'featureStack' Image.
        description=out_file_name,  # Description for the export task.
        bucket=OUTPUT_BUCKET,  # The destination Cloud Storage bucket.
        fileNamePrefix=FOLDER + '/' + out_file_name,  # File name prefix for the exported images.
        region=region.getInfo()['coordinates'],  # The bounding region for the export task.
        scale=SAMPLE_RESOLUTION,  # The spatial resolution in meters for the exported images.
        fileFormat='TFRecord',  # File format for the exported images (TensorFlow Record).
        maxPixels=1e10,  # Maximum number of pixels allowed for the export task.
        formatOptions={
            'patchDimensions': KERNEL_SHAPE,  # Dimensions of the image patches for segmentation.
            'kernelSize': kernel_buffer,  # Size of the kernel buffer for segmentation.
            'compressed': True,  # Enable compression for the exported images.
            'maxFileSize': 104857600  # Maximum file size for each exported image in bytes.
        }
    )
    print(FOLDER + '/' + out_file_name)
    # Start the export task.
    task.start()

    # Block until the task completes.
    print('Running image export to Cloud Storage...')
    import time
    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(30)

    # Check if the export task encountered an error.
    if task.status()['state'] != 'COMPLETED':
        print(f"Error with image export {task.status()['state']}")
    else:
        print('Image export completed.')

In [ ]:
doExport(predict_image, out_file_name, KERNEL_SIZE, pred_region)

**!remember to change user_folder to your own GEE username**

In [ ]:
# def doPrediction(out_image_base, in_image_bucket, out_image_bucket, user_folder, kernel_buffer):
#   """Perform inference on exported imagery, upload to Earth Engine.
#   """

#   print('Looking for TFRecord files...')

#   # Get a list of all the files in the output bucket.
#   filesList = !gsutil ls 'gs://'{in_image_bucket}'/'{FOLDER}

#   # Get only the files generated by the image export.
#   exportFilesList = [s for s in filesList if out_image_base in s]

#   # Get the list of image files and the JSON mixer file.
#   imageFilesList = []
#   jsonFile = None
#   for f in exportFilesList:
#     if f.endswith('.tfrecord.gz'):
#       imageFilesList.append(f)
#     elif f.endswith('.json'):
#       jsonFile = f

#   # Make sure the files are in the right order.
#   imageFilesList.sort()

#   from pprint import pprint
#   pprint(imageFilesList)
#   print(jsonFile)

#   import json
#   # Load the contents of the mixer file to a JSON object.
#   jsonText = !gsutil cat {jsonFile}
#   # Get a single string w/ newlines from the IPython.utils.text.SList
#   mixer = json.loads(jsonText.nlstr)
#   pprint(mixer)
#   patches = mixer['totalPatches']

#   # Get set up for prediction.
#   x_buffer = int(kernel_buffer[0] / 2)
#   y_buffer = int(kernel_buffer[1] / 2)

#   buffered_shape = [
#       KERNEL_SHAPE[0] + kernel_buffer[0],
#       KERNEL_SHAPE[1] + kernel_buffer[1]]

#   imageColumns = [
#     tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
#       for k in BANDS
#   ]

#   imageFeaturesDict = dict(zip(BANDS, imageColumns))

#   def parse_image(example_proto):
#     return tf.io.parse_single_example(example_proto, imageFeaturesDict)

#   def toTupleImage(inputs):
#     inputsList = [inputs.get(key) for key in BANDS]
#     stacked = tf.stack(inputsList, axis=0)
#     stacked = tf.transpose(stacked, [1, 2, 0])
#     return stacked

#    # Create a dataset from the TFRecord file(s) in Cloud Storage.
#   imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
#   imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
#   imageDataset = imageDataset.map(toTupleImage).batch(1)

#   # Perform inference.
#   print('Running predictions...')
#   predictions = m.predict(imageDataset, steps=patches, verbose=1)
#   # print(predictions[0])

#   print('Writing predictions...')
#   out_image_file = 'gs://' + out_image_bucket + '/' + FOLDER + '/' + out_image_base  + '.TFRecord'
#   writer = tf.io.TFRecordWriter(out_image_file)
#   patches = 0
#   for predictionPatch in predictions:
#     print('Writing patch ' + str(patches) + '...')
#     predictionPatch = predictionPatch[
#         x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

#     # Create an example.
#     example = tf.train.Example(
#       features=tf.train.Features(
#         feature={
#           'impervious': tf.train.Feature(
#               float_list=tf.train.FloatList(
#                   value=predictionPatch.flatten()))
#         }
#       )
#     )
#     # Write the example.
#     writer.write(example.SerializeToString())
#     patches += 1

#   writer.close()

#   # Start the upload.
#   out_image_asset = user_folder + '/' + out_image_base 
#   !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

for debug(change to your own username)

In [ ]:
user_folder = 'users/jindacz' # INSERT YOUR FOLDER HERE.
pred_kernel_buffer = [128, 128]
# date_start_str = '2019-07-01'
out_file_name = f'pred_Hanceville_{date_start_str}'
pred_region = ee.Geometry.Polygon(
    [[[-120.09501953125, 53.81667309827828], 
    [-120.09501953125, 56.69527587549462], 
    [-125.76396484375, 56.69527587549462],     
    [-125.76396484375, 53.81667309827828]]],  None, False)
FOLDER = 'fcnn-demo'

def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{OUTPUT_BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(f'imageFilesList {imageFilesList}')
  # print(f'jsonFile {jsonFile}')

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # print(f'jsonText{jsonText}')
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  # out_image_base = 'shufeitest123'
  out_image_file = 'gs://' + OUTPUT_BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  
  print(out_image_file)
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
    
  for predictionPatch in predictions:
    
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    
    # print(f'prediction patch {predictionPatch}')
    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    print('finish writing patch ' + str(patches) + '...')
    patches += 1


  #writer.close()
  print(f'fin writing')

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

# Run the prediction.
doPrediction(out_file_name, user_folder, pred_kernel_buffer, pred_region)

In [ ]:
# !python -m pip install tensorflow-io 
# !python -m pip uninstall -y tensorflow-io

In [ ]:
# doPrediction(out_file_name, OUTPUT_BUCKET, DATA_BUCKET, FOLDER, kernel_size)

In [ ]:
# # Run the prediction.
print(user_folder)
print(out_file_name)

In [ ]:
# out_image = ee.Image(user_folder + '/' + bj_image_base)
out_image = ee.Image(user_folder + '/' + out_file_name)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[53.81667309827828, -120.09501953125])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

export prediction to CSV

In [ ]:
pred_values = out_image.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=pred_region,
    scale=926.625433055833,  # The scale from the image's 'crs_transform'
)

impervious_list = pred_values.get('impervious').getInfo()

# Convert the list of pixel values to a pandas DataFrame
df = pd.DataFrame(impervious_list, columns=['prediction'])

# Save the DataFrame as a CSV file
df.to_csv('prediction_results.csv', index=False)

In [ ]:
print(pred_values.get('impervious'))

# NEWEST VER

code citation:
https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb#scrollTo=FyRpvwENxE-A

In [ ]:
# pip install earthengine-api --upgrade

In [ ]:
# pip install geemap

**remember to re-auth every time you want to upload**

In [ ]:
import ee
print()
ee.Authenticate()
ee.Initialize()

In [ ]:
import numpy as np
import pandas as pd

import geemap
import folium

In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.run_functions_eagerly(True)

Define datasets

In [ ]:
feature_list = []
# Fire label
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/FIRMS
fire_map_info = {'name':'FIRMS', 'band' :'T21', 'min': 300, 'max': 509.29}
# MODIS Normalized difference vegetation index NDVI dataset
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_MOD09GA_006_NDVI
veg_map_info = {'name':'MODIS/MOD09GA_006_NDVI', 'band' :'NDVI', 'min': -1, 'max': 1}
feature_list.append(veg_map_info)
# MODIS Leaf Area Index/FPAR 4-Day Global 500m
# 2002-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD15A3H
# upper limit is 20+, so set max = 30
leaf_map_info = {'name':'MODIS/061/MCD15A3H', 'band' :'Lai', 'min': 0, 'max': 30}
feature_list.append(leaf_map_info)
# FLDAS: Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System
# 1982-2023
# https://developers.google.com/earth-engine/datasets/catalog/NASA_FLDAS_NOAH01_C_GL_M_V001
soil_moist_map_info = {'name':'NASA/FLDAS/NOAH01/C/GL/M/V001', 'band' :'SoilMoi00_10cm_tavg', 'min': 0, 'max': 1}
feature_list.append(soil_moist_map_info)
# Terra Land Surface Temperature and Emissivity Daily Global 1km
# 2000-2023
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1
temp_map_info = {'name':'MODIS/061/MOD11A1', 'band' :'LST_Day_1km', 'min': 13658, 'max': 15658}
feature_list.append(temp_map_info)
# ERA5 Monthly Aggregates dataset
# 1979-2020
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY
precipitation_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'total_precipitation', 'min': 0, 'max': 0.4}
feature_list.append(precipitation_map_info)
wind_speed_u_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'u_component_of_wind_10m', 'min': -8.7, 'max': 8.7}
feature_list.append(wind_speed_u_map_info)
wind_speed_v_map_info = {'name':'ECMWF/ERA5/MONTHLY', 'band' :'v_component_of_wind_10m', 'min': -6.8, 'max': 6.8}
feature_list.append(wind_speed_v_map_info)

# Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3
# 2015-end2019
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global
land_cover_info = {'name':"COPERNICUS/Landcover/100m/Proba-V-C3/Global", 'band': 'discrete_classification', 'min': 0, 'max': 200}
# feature_list.append(wind_speed_v_map_info)
# print(feature_list)
print(len(feature_list))
print(type(feature_list))

Define label and feature bands

In [ ]:
LABEL = fire_map_info['band']
BANDS = []
for feature in feature_list:
  BANDS += [feature['band']]
FEATURE_NAMES = BANDS + [LABEL]
print(FEATURE_NAMES)

Define area of interest and time range

In [ ]:
# PROJECT = 'big-bliss-393615'
# DATA_BUCKET = 'big-bliss-393615-6140-data-bucket'
# OUTPUT_BUCKET = 'big-bliss-393615-6140-output-bucket'
PROJECT = 'ee-my-char'
DATA_BUCKET = '6140-data-bucket'
OUTPUT_BUCKET = '6140-output-bucket'
# Specify names locations for outputs in Cloud Storage.
FOLDER = 'fcnn-demo'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'
# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
# KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURE_NAMES]
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))
# Specify model training parameters.
BATCH_SIZE = 64
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

In [ ]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Canada = countries.filter(ee.Filter.eq('country_na', 'Canada')).geometry() # <class 'ee.geometry.Geometry'>
America = countries.filter(ee.Filter.eq('country_na', 'United States')).geometry()

In [ ]:
train_start_year = 2015
train_end_year = 2015
train_start_month = 4
train_end_month = 4
# predict fire 4 weeks in advance
fire_pred_range = 2
# every 2 weeks get one image
sample_gap = 2
train_region_geometry = America
pred_region_geometry = Canada

- How many time slices do we have Calculated in getTrainImageCollection

In [ ]:
total_temporal_slices = 0  # Initialize the total number of temporal slices.
temporal_list = []  # Initialize the list to store temporal date strings.

# Loop through the years and months within the specified training time range.
for yr in range(train_start_year, train_end_year + 1):
    for mo in range(train_start_month, train_end_month + 1):
        total_temporal_slices += sample_gap  # Increment the total time slices by 'sample_gap'.
        temporal_list.append(f'{yr}-{mo:02d}-01')  # Add the first day of the month to the list.
        temporal_list.append(f'{yr}-{mo:02d}-15')  # Add the fifteenth day of the month to the list.
print(temporal_list)

If 'is_label', advance the time range by 'fire_pred_range', e.g. we use first 0-15 as feature and 16-30 as label

In [ ]:
# Helper function avoid repeating
def getImageCollection(dataset, bounds, band):
  return ee.ImageCollection(dataset).filterBounds(bounds).select(band)

def getTrainImageCollection(map_info, is_label=False):
  img_collection = getImageCollection(map_info['name'],train_region_geometry, map_info['band']) # Get the image collection based on the provided map_info.
  # print(img_collection.size().getInfo())
  img_list = ee.List([])

  advance_range = 0 if not is_label else fire_pred_range
  for yr in range(train_start_year, train_end_year+1):
    for mo in range(train_start_month, train_end_month + 1):
      # first half of a month
      start = ee.Date(f'{yr}-{mo:02d}-01').advance(advance_range, 'week')
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end) # Filter the image collection based on the date range.
      if not is_label:
        med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
      if is_label:
        med = date_filtered.median().unmask(0).divide(300).round().byte() # credit: david
      print(med.getInfo())
      img_list = img_list.add(med)

      # second half of a month
      start = end
      end = start.advance(2, 'week')
      date_filtered = img_collection.filterDate(start, end) # Filter the image collection based on the date range.
      if date_filtered.size().getInfo() > 0:  # Check if there are images in the second half of the month.
        # print("value exists from 15")
        if not is_label:
          med = date_filtered.median().unitScale(map_info['min'], map_info['max']) # Calculate median and rescale.
        if is_label:
          med = date_filtered.median().unmask(0).divide(300).round().byte() # credit: david
        print(med.getInfo())
        
        # med = date_filtered.median().unmask(0).unitScale(map_info['min'], map_info['max'])
        # print(f'med type: {type(med)}')
      img_list = img_list.add(med)

  # print(img_list.size().getInfo())
  return img_list

Each element in feature_img_lists is a temporal imageCollection of a single band

In [ ]:
# Label
firms = getTrainImageCollection(fire_map_info, True)

`firms_mask_lists` stores firms info in temporal order
each element in the list is a list of 2 elements, the first is `has_fire_mask`, the second is `no_fire_mask`

In [ ]:
def get_firms_mask(firms):
    firms_masks = []
    for idx in range(firms.size().getInfo()):
        img = ee.Image(firms.get(idx))
        has_fire_mask = img.select('T21').eq(1)    
        no_fire_mask = img.select('T21').eq(0) 
        cur_time_slice_mask_set = []
        cur_time_slice_mask_set.append(has_fire_mask)
        cur_time_slice_mask_set.append(no_fire_mask)
        firms_masks.append(cur_time_slice_mask_set)
    return firms_masks
firms_mask_lists = get_firms_mask(firms)

In [ ]:
feature_img_lists = []
for feature in feature_list:
  img_list = getTrainImageCollection(feature)
  feature_img_lists.append(img_list)
# print(type(feature_img_lists))
# print(feature_img_lists)

In [ ]:
# `feature_img_masked_lists` stores lists of features
# each list is ordered in temporal order
# within each list, stores a list of 2 elements, the first is feature with fire mask, the second is feature with no fire mask
def get_features_with_firms_mask(feature_img_lists, firms_mask_list):
    feature_img_masked_lists = []
    for cur_feature_all_time in feature_img_lists:
        cur_feature_masked_list = []
        for cur_time_slice in range(cur_feature_all_time.size().getInfo()):
            img = ee.Image(cur_time_slice)
            img_mask_with_fire = img.updateMask(firms_mask_list[cur_time_slice][0])
            img_mask_no_fire = img.updateMask(firms_mask_list[cur_time_slice][1])
            cur_time_slice_mask_set = []
            cur_time_slice_mask_set.append(img_mask_with_fire)
            cur_time_slice_mask_set.append(img_mask_no_fire)
            cur_feature_masked_list.append(cur_time_slice_mask_set)
        feature_img_masked_lists.append(cur_feature_masked_list)
    return feature_img_masked_lists    

feature_img_masked_lists = get_features_with_firms_mask(feature_img_lists, firms_mask_lists)

export sample unmask fire as csv

In [ ]:
# # date_start = '2015-04-01'
# # date_end = '2015-12-01'
# # dataset = ee.ImageCollection('FIRMS').filter(
# #     ee.Filter.date(date_start, date_end))
# # fires = dataset.select('T21').first().unmask(0)
# Map = geemap.Map(center=[47.295, -119.086], zoom=6)
# # Map
# fires = ee.Image(firms.get(1))
# print(type(fires))

# fire_values = fires.reduceRegion(
#     reducer=ee.Reducer.toList(),
#     geometry=Map.getBounds(False),
#     scale=1000,  # The scale from the image's 'crs_transform'
#     maxPixels=90000000
# )
# fire_values.getInfo()
# df = pd.DataFrame(fire_values.getInfo())
# df.to_csv('fires_data.csv', index=False)

test unmask

In [ ]:
# import pandas as pd

# df = pd.read_csv('fires_data.csv')

# unique_values_count = df.nunique()

# unique_values = {}
# for column in df.columns:
#     unique_values[column] = df[column].unique()

#     print(f"Unique values in column '{column}':")
#     print(unique_values[column])
#     print(f"Number of unique values: {len(unique_values[column])}")

#     value_counts = df[column].value_counts()
#     print(f"Value counts:")
#     print(value_counts)
#     print()

Debug function: get min & max by iterating through all pixels

In [ ]:
# @title
# For debug, can skip
def get_min(gee_image, key):
  min_dic = gee_image.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry = Canada.geometry(),
    crs =  'EPSG:4326',
    crsTransform = [1, 0, 0, 0, 1, 0]
  )
  return min_dic.getInfo().get(key)
 
def get_max(gee_image, key):
  max_dic = gee_image.reduceRegion(
    reducer= ee.Reducer.max(),
    geometry = Canada.geometry(),
    crs =  'EPSG:4326',
    crsTransform = [1, 0, 0, 0, 1, 0]
  )
  return max_dic.getInfo().get(key)

Visualize all maps

In [ ]:
# # @title
# # visualization
# # normalized
# firms_vis = firms.median().clip(train_region_geometry)
# vegetation_vis = vegetation.median().clip(train_region_geometry)
# leaf_vis = leaf_index.median().clip(train_region_geometry)
# soil_moist_vis = soil_moist.median().clip(train_region_geometry)
# precipitation_vis = precipitation.median().clip(train_region_geometry)
# temperature_vis = temperature.median().clip(train_region_geometry)
# wind_speed_vis = wind_speed_u.median().clip(train_region_geometry).cat(wind_speed_v.median().clip(train_region_geometry))

# Map = geemap.Map()
# ROI = ee.Geometry.Point([-122.6544033634249, 48.99021037738678]).buffer(100000);

# Map.addLayer(firms_vis, {'min': 0, 'max': 1, 'palette': ['LightYellow','Yellow', 'Gold', 'Orange', 'Tomato', 'Red']}, 'FIRMS')
# Map.addLayer(vegetation_vis, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, 'vegetation')
# Map.addLayer(leaf_vis, {'min': 0, 'max': 1, 'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']}, 'leaf index')
# Map.addLayer(soil_moist_vis, {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'soil moisture')
# Map.addLayer(wind_speed_vis, {'min': 0, 'max': 1, 'palette': ['blue', 'red']}, 'wind speed')
# Map.addLayer(precipitation_vis, {'min': 0,
#   'max': 1,
#    'palette': [
#     '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff',
#     '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00',
#     'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff',
#   ]}, 'precipitatin')

# Map.addLayer(temperature_vis, {  'min': 0,
#   'max': 1,
#   'palette': [
#     '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
#     '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
#     '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
#     'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
#     'ff0000', 'de0101', 'c21301', 'a71001', '911003'
#   ]}, 'temperature')

# Map.addLayerControl()
# Map

In [ ]:
# # @title
# Map.user_roi

Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band. This is a key step that bears emphasis: to export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.

In [ ]:
list_1d = ee.List.repeat(1, KERNEL_SIZE)
list_2d = ee.List.repeat(list_1d, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, list_2d)

In [ ]:
def mask_stack_kernel(feature_stack, mask, kernel):
    # mask
    masked_feature_stack = feature_stack.updateMask(mask)
    # stack, no need to do because mask step adds the mask band to result
    # masked_feature_stack = ee.Image.cat([masked_feature_stack, mask])
    # kernel
    arrays = masked_feature_stack.neighborhoodToArray(kernel)
    return arrays

def get_feature_stack(kernel, feature_img_lists, firms_mask_list):
    list_arrays_has_fire = []
    list_arrays_no_fire = []
    for tm in range(total_temporal_slices): 
        # Get the image for the current temporal slice from feature_img_lists
        featureStack = feature_img_lists[0].get(tm)
        # Concatenate all the bands sampled from different images into featureStack
        for idx in range(1, len(feature_img_lists)):
            # Combine all bands sampled from the same time to featureStack
            featureStack = ee.Image.cat([featureStack, feature_img_lists[idx].get(tm)]).float()
        
        arrays_has_fire = mask_stack_kernel(featureStack, firms_mask_list[tm][0], kernel)
        arrays_no_fire = mask_stack_kernel(featureStack, firms_mask_list[tm][1], kernel)
        print(f'time = {tm}:\nhas fire:{arrays_has_fire.getInfo()}\nno fire:{arrays_no_fire.getInfo()} ')
        list_arrays_has_fire += [arrays_has_fire]
        list_arrays_no_fire += [arrays_no_fire]
    
    return list_arrays_has_fire, list_arrays_no_fire

list_arrays_has_fire, list_arrays_no_fire = get_feature_stack(kernel, feature_img_lists, firms_mask_lists)
print(f'{len(list_arrays_has_fire)}, {len(list_arrays_no_fire)}')

feature_img_lists
NDVI:[4-1, ..., 12-1]
LAI :[4-1, ..., 12-1]

list_arrays [4-1, ...,]

In [ ]:
# list = ee.List.repeat(1, KERNEL_SIZE)
# lists = ee.List.repeat(list, KERNEL_SIZE)
# kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

# list_arrays = []

# for tm in range(total_temporal_slices): # 2
#   # Get the image for the current temporal slice from feature_img_lists
#   featureStack = feature_img_lists[0].get(tm)
#   # Concatenate all the bands sampled from different images into featureStack
#   for idx in range(1, len(feature_img_lists)):
#     # Combine all bands sampled from the same time to featureStack
#     featureStack = ee.Image.cat([featureStack, feature_img_lists[idx].get(tm)]).float()
#   # Concatenate the FIRMS band to featureStack and convert it to float
#   featureStack = ee.Image.cat([featureStack, firms.get(tm)])
#   # Apply neighborhood operations using the kernel and convert to arrays
#   arrays = featureStack.neighborhoodToArray(kernel)
#   # Add the array to the list_arrays for the current temporal slice
#   list_arrays += [arrays]

In [ ]:
# # Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
# states = ee.FeatureCollection("TIGER/2016/States")
# # Get the distinct values of the 'NAME' property
# distinct_names = states.distinct('NAME').aggregate_array('NAME')
# # Convert the distinct names to a Python list
# all_state_names = distinct_names.getInfo()
# # Print all state names
# print(all_state_names)

Get polygons for training and evaluation

In [ ]:
# Create an empty FeatureCollection to store the training polygons
trainingPolys = ee.FeatureCollection([])

# Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
states = ee.FeatureCollection("TIGER/2016/States")

# List of state names to be used for training
# state_list = ['California', 'Oregon','Montana','Washington','Arizona', 'Idaho','Texas','Kansas','Nevada','New Mexico']
state_list = ['California', 'Oregon', 'Texas', 'Arizona', 'Nevada', 'New Mexico']
# Iterate through each state in 'state_list'
for state in state_list:
    # Merge the geometry of the state from 'states' into 'trainingPolys'
    trainingPolys = trainingPolys.merge(states.filter(ee.Filter.eq('NAME', state)).geometry())

# Convert the 'trainingPolys' FeatureCollection to a List
trainingPolysList = trainingPolys.toList(trainingPolys.size())
# print(type(trainingPolysList))

In [ ]:
# Create an empty FeatureCollection to store the training polygons
valPolys = ee.FeatureCollection([])

# Load the "TIGER/2016/States" dataset as the 'states' FeatureCollection
states = ee.FeatureCollection("TIGER/2016/States")

# List of state names to be used for training
# state_list = ['Oklahoma', 'Colorado']
state_list = ['Wyoming', 'Idaho', 'Alaska', 'Washington', 'New Jersey', 'Pennsylvania']

# Iterate through each state in 'state_list'
for state in state_list:
    # Merge the geometry of the state from 'states' into 'trainingPolys'
    valPolys = valPolys.merge(states.filter(ee.Filter.eq('NAME', state)).geometry())
    # print(states.filter(ee.Filter.eq('NAME', state)).geometry().getInfo())

# Convert the 'trainingPolys' FeatureCollection to a List
valPolysList = valPolys.toList(valPolys.size())

# print(valPolysList.size().getInfo())
print(type(valPolysList))
# # print(valPolys.geometry().getInfo())
# print(valPolys.get(0))

Visualize training and evaluation regions on map

In [ ]:
Map = geemap.Map()
Map.addLayer(trainingPolys, {'color': 'FF0000'})
Map.addLayer(valPolys, {'color': '00FF00'})
Map

- Export training data

In [ ]:
SAMPLE_NUM_IN_POLYGON = 200 # Number of shards in each polygon.
SAMPLE_PIXEL_IN_POLYGON = 2000 # Total sample pixels in each polygon.

In [ ]:
# Sizes of the training and evaluation datasets.
TRAIN_SIZE = len(temporal_list) * trainingPolysList.size().getInfo() * SAMPLE_NUM_IN_POLYGON
EVAL_SIZE = len(temporal_list) * valPolysList.size().getInfo() * SAMPLE_NUM_IN_POLYGON
# print(TRAIN_SIZE)

In [ ]:
SAMPLE_RESOLUTION = 1000 # 1km

In [ ]:
def export_data(file_prefix, sample_source_list,file_suffix, num_sample_pixel, is_nofire=False):
    for idx, time_str in enumerate(temporal_list): # 18, Iterate through each time string in 'temporal_list'
        for g in range(trainingPolysList.size().getInfo()): # 10, Iterate through each geometry in 'trainingPolysList'
            geomSample = ee.FeatureCollection([])
            for i in range(SAMPLE_NUM_IN_POLYGON): # 200
                sample = sample_source_list[idx].sample(
                    region = ee.Feature(trainingPolysList.get(g)).geometry(),
                    scale = SAMPLE_RESOLUTION, # 1km
                    numPixels = num_sample_pixel / SAMPLE_NUM_IN_POLYGON, # 200/200, 
                    seed = i,
                    tileScale = 8 # make your process faster
                )
                geomSample = geomSample.merge(sample)
            desc = f'{file_prefix}_{time_str}_{str(g)}_{file_suffix}'
            print(desc)
            task = ee.batch.Export.table.toCloudStorage(
              collection = geomSample,
              description = desc,
              bucket = DATA_BUCKET,
              fileNamePrefix = FOLDER + '/' + desc,
              fileFormat = 'TFRecord',
              selectors = FEATURE_NAMES
            )
            task.start()

In [ ]:
def export_data_eval(file_prefix, sample_source_list,file_suffix, num_sample_pixel, is_nofire=False):
    for idx, time_str in enumerate(temporal_list): # 18, Iterate through each time string in 'temporal_list'
        for g in range(valPolysList.size().getInfo()): # 10, Iterate through each geometry in 'trainingPolysList'
            geomSample = ee.FeatureCollection([])
            for i in range(SAMPLE_NUM_IN_POLYGON): # 200
                sample = sample_source_list[idx].sample(
                    region = ee.Feature(valPolysList.get(g)).geometry(),
                    scale = SAMPLE_RESOLUTION, # 1km
                    numPixels = num_sample_pixel / SAMPLE_NUM_IN_POLYGON, # 200/200, 
                    seed = i,
                    tileScale = 8 # make your process faster
                )
                geomSample = geomSample.merge(sample)
            desc = f'{file_prefix}_{time_str}_{str(g)}_{file_suffix}'
            print(desc)
            task = ee.batch.Export.table.toCloudStorage(
              collection = geomSample,
              description = desc,
              bucket = DATA_BUCKET,
              fileNamePrefix = FOLDER + '/' + desc,
              fileFormat = 'TFRecord',
              selectors = FEATURE_NAMES
            )
            task.start()

In [ ]:
NUM_PIXEL_SAMPLE_NO_FIRE = 200
NUM_PIXEL_SAMPLE_FIRE = NUM_PIXEL_SAMPLE_NO_FIRE * 1000
print(NUM_PIXEL_SAMPLE_FIRE, NUM_PIXEL_SAMPLE_NO_FIRE)

In [ ]:
# export_data(TRAINING_BASE, list_arrays_has_fire, 'fire', NUM_PIXEL_SAMPLE_FIRE)   
# export_data(TRAINING_BASE, list_arrays_no_fire, 'nofire', NUM_PIXEL_SAMPLE_NO_FIRE, is_nofire=True)           

In [ ]:
# export_data_eval(EVAL_BASE, list_arrays_has_fire, 'fire', NUM_PIXEL_SAMPLE_FIRE)
# export_data_eval(EVAL_BASE, list_arrays_no_fire, 'nofire', NUM_PIXEL_SAMPLE_FIRE, is_nofire=True)           

Test: use stratified result of firms as sampleRegion input for features

In [ ]:
# firms_stratified_time_result = [] # list of every time slice
# for i in range(firms.size().getInfo()): # firms.size() == time slices
#     cur_firm = ee.Image(firms.get(i))
#     firms_stratified_region_result = [] # list of every region in trainingPolysList
    
#     for g in range(trainingPolysList.size().getInfo()):
#         firms_stratified_feature_collection = ee.FeatureCollection([]) # feature collection at current region and time slice
#         for sample_idx in range(SAMPLE_NUM_IN_POLYGON): # Loop through each sample index for the current polygon
#             sample = cur_firm.stratifiedSample(
#                 numPoints=SAMPLE_PIXEL_IN_POLYGON / SAMPLE_NUM_IN_POLYGON, # 5
#                 classBand='T21',
#                 region = ee.Feature(trainingPolysList.get(g)).geometry(),
#                 scale = SAMPLE_RESOLUTION, # 1km
#                 seed = i,
#                 tileScale = 8, # make your process faster
#                 geometries = True
#             )
#             firms_stratified_feature_collection = firms_stratified_feature_collection.merge(sample)
#         # print(firms_stratified_feature_collection.distinct("T21").getInfo())
#         # print(firms_stratified_feature_collection.size().getInfo())
#         firms_stratified_region_result.append(firms_stratified_feature_collection)
#     firms_stratified_time_result.append(firms_stratified_region_result)
    
# # print(type(firms_stratified_time_result[0][0])) # <class 'ee.featurecollection.FeatureCollection'>

In [ ]:
# for time_idx, time_slice_result in enumerate(firms_stratified_time_result): #2
#     # print(len(time_slice_result))
#     for region_idx, region_result in enumerate(time_slice_result): # 2
#         print(region_result.size().getInfo()) # featurecollection
#         print(type( list_arrays[time_idx]))
#         geomSample = list_arrays[time_idx].sampleRegions(
#             collection = region_result,
#             properties = ee.List(['T21']),
#             scale = SAMPLE_RESOLUTION, # 1km
#             tileScale = 8 # make your process faster
#           )
#         # print(type(geomSample))
#         desc = f'sampleRegion_{time_idx}_{region_idx}'
#         # print(desc)
#         task = ee.batch.Export.table.toCloudStorage(
#           collection = geomSample,
#           description = desc,
#           bucket = DATA_BUCKET,
#           fileNamePrefix = FOLDER + '/' + desc,
#           fileFormat = 'TFRecord',
#           selectors = FEATURE_NAMES
#         )
#         # task.start()

sample and export train

stratify sample

- export eval

Load the data exported
去到bucket已经能看到这个file下载好了! 在那边! 接下来就要读 **Text fett markieren**

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  # print(f'Example Proto: {example_proto}')
  parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
  # print(f'Parsed features: {parsed_features}')
  # print()
  return parsed_features

def to_tuple(inputs, deb=True):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURE_NAMES]
  stacked = tf.stack(inputsList, axis=0)

  if deb:
    print(f'to_tuple inputsList {inputsList}')
    print(f'to_tuple len list {len(inputsList)}')
    print(f'to_tuple stacked shape {stacked.shape}')

  # Convert from CHW to HWC
  # See https://caffe2.ai/docs/tutorial-image-pre-processing.html
  stacked = tf.transpose(stacked, [1, 2, 0])

  if deb:
    print(f'to_tuple after reshape {stacked.shape}')
    print(f'length of BANDS??? {len(BANDS)}')
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]

In [ ]:
# visualize tfrecord
import matplotlib.pyplot as plt

def visualize_dataset(dataset):
  for batch in dataset.take(3):
    # print(f'Batch keys: {batch.keys()}')
    # print(f'Batch data types: {[type(v.numpy()) for v in batch.values()]}')
    f, axarr = plt.subplots(4, 2)
    rows = np.repeat([0, 1, 2, 3], 2)
    cols = np.repeat([[0, 1]], 4, axis=0).ravel()
    for v, r, c in zip(batch.values(), rows, cols):
      axarr[r, c].imshow(v, cmap='gray')

def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  # print(f'get_dataset:parse step1 {dataset}')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  # print(f'get_dataset step1-2 {dataset.element_spec}')
  # for batch in dataset.take(1):
  #   print("Batch keys:", batch.keys())
  #   for key, value in batch.items():
  #       print(f"Key: {key}, Shape: {value.shape}, Data Type: {value.dtype}")
    
  # visualize_dataset(dataset)
  # print(f'get_dataset step2 {dataset}')
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  # print(f'get_dataset step3 {dataset}')
  return dataset

train, eval

fixed problem get training dataset and eval dataset

get new dataset after stratify

In [ ]:
def get_training_dataset(filename_pattern):
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = filename_pattern
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset


def get_eval_dataset(filename_pattern):
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = filename_pattern
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

In [ ]:
training_nofire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*' + '*_nofire*')

In [ ]:
training_fire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*' + '*_fire*')

In [ ]:
eval_fire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*' + '*_fire*')

In [ ]:
eval_nofire = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*' + '*_nofire*')

In [ ]:
training = get_training_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'training*')

In [ ]:
evaluation = get_eval_dataset('gs://' + DATA_BUCKET + '/' + FOLDER + '/' + 'eval*')

unet

In [ ]:
filter_1 = 32
filter_2 = 4

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.models as models
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers

In [ ]:
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model(filter_base):
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, filter_base) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, filter_base*2) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, filter_base*4) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, filter_base*8) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, filter_base*16) # 8
	center = conv_block(encoder4_pool, filter_base*32) # center
	decoder4 = decoder_block(center, encoder4, filter_base*16) # 16
	decoder3 = decoder_block(decoder4, encoder3, filter_base*8) # 32
	decoder2 = decoder_block(decoder3, encoder2, filter_base*4) # 64
	decoder1 = decoder_block(decoder2, encoder1, filter_base*2) # 128
	decoder0 = decoder_block(decoder1, encoder0, filter_base) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS],
		run_eagerly=True
		)

	return model

In [ ]:
# import tensorflow.keras.layers as layers
# import tensorflow.keras.losses as losses
# import tensorflow.keras.models as models
# import tensorflow.keras.metrics as metrics
# import tensorflow.keras.optimizers as optimizers

# def conv_block(input_tensor, num_filters):
#     encoder = layers.Conv2D(num_filters, (3, 3), padding='same', activation='relu')(input_tensor)
#     encoder = layers.BatchNormalization()(encoder)
#     return encoder

# def encoder_block(input_tensor, num_filters):
#     encoder = conv_block(input_tensor, num_filters)
#     encoder_pool = layers.MaxPooling2D((2, 2))(encoder)
#     return encoder_pool, encoder

# def decoder_block(input_tensor, concat_tensor, num_filters):
#     decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', activation='relu')(input_tensor)
#     decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
#     decoder = layers.BatchNormalization()(decoder)
#     return decoder

# def get_model():
#     inputs = layers.Input(shape=[None, None, len(BANDS)])
#     encoder0_pool, encoder0 = encoder_block(inputs, 32)
#     encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
#     encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
#     center = conv_block(encoder2_pool, 256)  # Center block with reduced filters
#     decoder2 = decoder_block(center, encoder2, 128)
#     decoder1 = decoder_block(decoder2, encoder1, 64)
#     decoder0 = decoder_block(decoder1, encoder0, 32)
#     outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

#     model = models.Model(inputs=[inputs], outputs=[outputs])
    
#     model.compile(
# 		optimizer=optimizers.get(OPTIMIZER),
# 		loss=losses.get(LOSS),
# 		metrics=[metrics.get(metric) for metric in METRICS],
# 		run_eagerly=True
#     )

#     return model

In [ ]:
# taken from https://github.dev/pandyamarut/HalfUnet-ImageSegmentation

import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.models as models
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers

def ghost_module(inputs):
    conv1 = layers.Conv2D(32, 3, padding='same', kernel_initializer='he_normal')(inputs)
    batch1 = layers.BatchNormalization()(conv1)
    act1 = layers.Activation('relu')(batch1)
    conv2 = layers.SeparableConv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(act1)
    return layers.concatenate([act1, conv2], axis=3)

def half_unet_model(input_shape=(256, 256, 7)):

    inputs = layers.Input(shape=input_shape, name="image")
    x1 = ghost_module(ghost_module(inputs))
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
    x2 = ghost_module(ghost_module(pool1))
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
    x3 = ghost_module(ghost_module(pool2))
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(x3)
    x4 = ghost_module(ghost_module(pool3))
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(x4)
    x5 = ghost_module(ghost_module(pool4))

    up5 = layers.UpSampling2D(size=(input_shape[0] // x5.shape[1], input_shape[1] // x5.shape[2]))(x5)
    up4 = layers.UpSampling2D(size=(input_shape[0] // x4.shape[1], input_shape[1] // x4.shape[2]))(x4)
    up3 = layers.UpSampling2D(size=(input_shape[0] // x3.shape[1], input_shape[1] // x3.shape[2]))(x3)
    up2 = layers.UpSampling2D(size=(input_shape[0] // x2.shape[1], input_shape[1] // x2.shape[2]))(x2)

    upScaled = layers.Add()([x1, up2, up3, up4, up5])
    all_conv = ghost_module(ghost_module(upScaled))
    final_conv = layers.Conv2D(1, 1, activation='sigmoid')(all_conv)

    model = models.Model(inputs=[inputs], outputs=[final_conv], name="Half-UNet")
    model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS],
		run_eagerly=True
    )
    return model

sanity check

In [ ]:
# # Sizes of the training and evaluation datasets.
# TRAIN_SIZE = 16000
# EVAL_SIZE = 8000
# BATCH_SIZE = 64
# print(type(training))
# print(EPOCHS)
# print(TRAIN_SIZE)
# print(BATCH_SIZE)
# print(int(TRAIN_SIZE / BATCH_SIZE))
# # print(len(list(training)))
# print(EVAL_SIZE)
# # len(training)
# # Check if GPU is available
# if tf.test.is_gpu_available():
#     # Get the list of available GPU devices
#     gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#     if gpu_devices:
#         # Print the name of the first GPU device
#         print("Using GPU:", gpu_devices[0].name)
#     else:
#         print("No GPU devices available.")
# print("Available GPU devices:")
# print(tf.config.list_physical_devices('GPU'))
# print("Current device used for computation:")
# print(tf.config.list_logical_devices('GPU'))

In [ ]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

modular programming: make sure everything right before training

In [ ]:
stop

model training visualization

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

class TQDMCallback(tf.keras.callbacks.Callback):
    def __init__(self, total_epochs):
        super(TQDMCallback, self).__init__()
        self.total_epochs = total_epochs
        self.pbar = tqdm(total=self.total_epochs, unit='epoch')

    def on_epoch_end(self, epoch, logs=None):
        self.pbar.update(1)
        self.pbar.set_description(f"Epoch {epoch + 1}/{self.total_epochs}, Loss: {logs['loss']:.4f}, "
                                  f"RMSE: {logs['root_mean_squared_error']:.4f}, ")
                                  # f"Val_Loss: {logs['val_loss']:.4f}, "
                                  # f"Val_RMSE: {logs['val_root_mean_squared_error']:.4f}")

    def on_train_end(self, logs=None):
        self.pbar.close()
tqdm_callback = TQDMCallback(total_epochs=EPOCHS)

In [ ]:
# m = get_model(filter_2)
m = half_unet_model()

see model parameter

In [ ]:
m.summary()

In [ ]:
history = m.fit(
    x=training,
    epochs=10,
    # steps_per_epoch=int(16000 / 1),
    steps_per_epoch=int(16000 / 64),
    validation_data=evaluation,
    validation_steps=8000,
    callbacks=[tqdm_callback],
)

In [ ]:
m.summary()

Successful fit: 
    model is simplified into containing only a few layers

In [ ]:
history = m.fit(
    x=training,
    epochs=10,
    # steps_per_epoch=int(16000 / 1),
    steps_per_epoch=int(16000 / 64),
    validation_data=evaluation,
    validation_steps=8000,
    callbacks=[tqdm_callback],
)

save model

In [ ]:
m.save('saved_model.h5')

load model

In [ ]:
from tensorflow.keras.models import load_model
m = load_model('saved_model.h5')

In [ ]:
print(history.params)
print(history.history.keys())

visualize

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Training and Validation RMSE')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
batch_size = 64


ev = evaluation.take(100)
# Predict on the evaluation dataset with the specified number of steps
predictions = m.predict(x=ev)

print(len(predictions[0]))
print(len(predictions[0][0]))

In [ ]:
# threshold = 0.5
# predicted_labels = np.where(predictions > threshold, 1, 0)

# true_labels = np.array([label for _, label in ev])

# accuracy = np.sum(predicted_labels == true_labels) / (len(true_labels) * 256*256)
# print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# # Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
# MODEL_DIR = 'gs://ee-docs-demos/fcnn-demo/trainer/model'
# m = tf.keras.models.load_model(MODEL_DIR)
# m.summary()

select image to export

In [ ]:
# print(featureStack.bandNames().getInfo())

In [ ]:
feature_img_lists = []
date_start_str = '2019-07-01'
pred_start = ee.Date(date_start_str)
pred_end = pred_start.advance(4, 'week')

for feature in feature_list:
  img_col = getImageCollection(feature['name'],pred_region_geometry, feature['band'])
  pred_img = img_col.filterDate(pred_start, pred_end).median().unitScale(feature['min'], feature['max'])
  feature_img_lists += [pred_img]
print(feature_img_lists)

In [ ]:
predict_image = feature_img_lists[0]
for idx in range(1, len(feature_img_lists)):
  # Combine all bands sampled from the same time to featureStack
  predict_image = ee.Image.cat([predict_image, feature_img_lists[idx]]).float()
# predict_image

In [ ]:
def doExport(predict_image, out_file_name, kernel_buffer, region):
    """Run the image export task.  Block until complete.
    """
    # Define the export task using the Earth Engine Python API.
    task = ee.batch.Export.image.toCloudStorage(
        image=predict_image,  # Select specific bands from the 'featureStack' Image.
        description=out_file_name,  # Description for the export task.
        bucket=OUTPUT_BUCKET,  # The destination Cloud Storage bucket.
        fileNamePrefix=FOLDER + '/' + out_file_name,  # File name prefix for the exported images.
        region=region.getInfo()['coordinates'],  # The bounding region for the export task.
        scale=SAMPLE_RESOLUTION,  # The spatial resolution in meters for the exported images.
        fileFormat='TFRecord',  # File format for the exported images (TensorFlow Record).
        maxPixels=1e10,  # Maximum number of pixels allowed for the export task.
        formatOptions={
            'patchDimensions': KERNEL_SHAPE,  # Dimensions of the image patches for segmentation.
            'kernelSize': kernel_buffer,  # Size of the kernel buffer for segmentation.
            'compressed': True,  # Enable compression for the exported images.
            'maxFileSize': 104857600  # Maximum file size for each exported image in bytes.
        }
    )
    print(FOLDER + '/' + out_file_name)
    # Start the export task.
    task.start()

    # Block until the task completes.
    print('Running image export to Cloud Storage...')
    import time
    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(30)

    # Check if the export task encountered an error.
    if task.status()['state'] != 'COMPLETED':
        print(f"Error with image export {task.status()['state']}")
    else:
        print('Image export completed.')

In [ ]:
doExport(predict_image, out_file_name, KERNEL_SIZE, pred_region)

**!remember to change user_folder to your own GEE username**

In [ ]:
# def doPrediction(out_image_base, in_image_bucket, out_image_bucket, user_folder, kernel_buffer):
#   """Perform inference on exported imagery, upload to Earth Engine.
#   """

#   print('Looking for TFRecord files...')

#   # Get a list of all the files in the output bucket.
#   filesList = !gsutil ls 'gs://'{in_image_bucket}'/'{FOLDER}

#   # Get only the files generated by the image export.
#   exportFilesList = [s for s in filesList if out_image_base in s]

#   # Get the list of image files and the JSON mixer file.
#   imageFilesList = []
#   jsonFile = None
#   for f in exportFilesList:
#     if f.endswith('.tfrecord.gz'):
#       imageFilesList.append(f)
#     elif f.endswith('.json'):
#       jsonFile = f

#   # Make sure the files are in the right order.
#   imageFilesList.sort()

#   from pprint import pprint
#   pprint(imageFilesList)
#   print(jsonFile)

#   import json
#   # Load the contents of the mixer file to a JSON object.
#   jsonText = !gsutil cat {jsonFile}
#   # Get a single string w/ newlines from the IPython.utils.text.SList
#   mixer = json.loads(jsonText.nlstr)
#   pprint(mixer)
#   patches = mixer['totalPatches']

#   # Get set up for prediction.
#   x_buffer = int(kernel_buffer[0] / 2)
#   y_buffer = int(kernel_buffer[1] / 2)

#   buffered_shape = [
#       KERNEL_SHAPE[0] + kernel_buffer[0],
#       KERNEL_SHAPE[1] + kernel_buffer[1]]

#   imageColumns = [
#     tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
#       for k in BANDS
#   ]

#   imageFeaturesDict = dict(zip(BANDS, imageColumns))

#   def parse_image(example_proto):
#     return tf.io.parse_single_example(example_proto, imageFeaturesDict)

#   def toTupleImage(inputs):
#     inputsList = [inputs.get(key) for key in BANDS]
#     stacked = tf.stack(inputsList, axis=0)
#     stacked = tf.transpose(stacked, [1, 2, 0])
#     return stacked

#    # Create a dataset from the TFRecord file(s) in Cloud Storage.
#   imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
#   imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
#   imageDataset = imageDataset.map(toTupleImage).batch(1)

#   # Perform inference.
#   print('Running predictions...')
#   predictions = m.predict(imageDataset, steps=patches, verbose=1)
#   # print(predictions[0])

#   print('Writing predictions...')
#   out_image_file = 'gs://' + out_image_bucket + '/' + FOLDER + '/' + out_image_base  + '.TFRecord'
#   writer = tf.io.TFRecordWriter(out_image_file)
#   patches = 0
#   for predictionPatch in predictions:
#     print('Writing patch ' + str(patches) + '...')
#     predictionPatch = predictionPatch[
#         x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

#     # Create an example.
#     example = tf.train.Example(
#       features=tf.train.Features(
#         feature={
#           'impervious': tf.train.Feature(
#               float_list=tf.train.FloatList(
#                   value=predictionPatch.flatten()))
#         }
#       )
#     )
#     # Write the example.
#     writer.write(example.SerializeToString())
#     patches += 1

#   writer.close()

#   # Start the upload.
#   out_image_asset = user_folder + '/' + out_image_base 
#   !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

for debug(change to your own username)

In [ ]:
user_folder = 'users/jindacz' # INSERT YOUR FOLDER HERE.
pred_kernel_buffer = [128, 128]
# date_start_str = '2019-07-01'
out_file_name = f'pred_Hanceville_{date_start_str}'
pred_region = ee.Geometry.Polygon(
    [[[-120.09501953125, 53.81667309827828], 
    [-120.09501953125, 56.69527587549462], 
    [-125.76396484375, 56.69527587549462],     
    [-125.76396484375, 53.81667309827828]]],  None, False)
FOLDER = 'fcnn-demo'

def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{OUTPUT_BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(f'imageFilesList {imageFilesList}')
  # print(f'jsonFile {jsonFile}')

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # print(f'jsonText{jsonText}')
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  # out_image_base = 'shufeitest123'
  out_image_file = 'gs://' + OUTPUT_BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  
  print(out_image_file)
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
    
  for predictionPatch in predictions:
    
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    
    # print(f'prediction patch {predictionPatch}')
    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    print('finish writing patch ' + str(patches) + '...')
    patches += 1


  #writer.close()
  print(f'fin writing')

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

# Run the prediction.
doPrediction(out_file_name, user_folder, pred_kernel_buffer, pred_region)

In [ ]:
# !python -m pip install tensorflow-io 
# !python -m pip uninstall -y tensorflow-io

In [ ]:
# doPrediction(out_file_name, OUTPUT_BUCKET, DATA_BUCKET, FOLDER, kernel_size)

In [ ]:
# # Run the prediction.
print(user_folder)
print(out_file_name)

In [ ]:
# out_image = ee.Image(user_folder + '/' + bj_image_base)
out_image = ee.Image(user_folder + '/' + out_file_name)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[53.81667309827828, -120.09501953125])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

export prediction to CSV

In [ ]:
pred_values = out_image.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=pred_region,
    scale=926.625433055833,  # The scale from the image's 'crs_transform'
)

impervious_list = pred_values.get('impervious').getInfo()

# Convert the list of pixel values to a pandas DataFrame
df = pd.DataFrame(impervious_list, columns=['prediction'])

# Save the DataFrame as a CSV file
df.to_csv('prediction_results.csv', index=False)

In [ ]:
print(pred_values.get('impervious'))